In [71]:
import json
import os

In [72]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [73]:
llm.invoke("What is latest news on Amazon Stock")

AIMessage(content="I don't have real-time data access to provide the latest news on Amazon stock or any other financial information. For the most current updates on Amazon's stock, I recommend checking financial news websites, stock market apps, or financial news channels. You can also look at Amazon's stock performance on platforms like Yahoo Finance, Google Finance, or Bloomberg.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 14, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cwb0mcHL4iMMeqLGIH1mZWQzfTcdb', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019ba9dc-0a28-7651-82b1-462a824

In [74]:
from openai import OpenAI
client = OpenAI()
system_instructions='You are an expert of sarcasm and provide responses accordingly.'
user_query="What is latest news on Amazon Stock"
def call_llm(system_instructions, user_query):
    response = client.responses.create(
    model="gpt-4o-mini",
    instructions=system_instructions,
    input=user_query
    )
    return response.output_text

In [75]:
system_instructions='You are an expert of sercasm and provide responses accordingly.'
response=call_llm(system_instructions, user_query)

In [76]:
print(response)

Oh, I'm sure Amazon's stock has either skyrocketed to the moon or plummeted into the abyss—because, you know, that's the only two options with stocks these days! Just Google "Amazon stock news," and you’ll find all the drama you need. Spoiler alert: Someone probably got rich or lost a fortune!


# Define the system prompt
# This prompt is used to guide the model's behavior.
# It tells the model that it is a tool-using assistant and that it must respond with ONLY valid JSON (no markdown, no explanation).
# It also defines the schema for the tool call JSON that the model will output.

# here we are saying to follow strcit json and providing format of json that model will output
# and we are also providing available tools and their signatures
# which will help model to create argument properly

In [77]:
system_instructions=f""""
You are tool-using assistant. 
You must response ONLY in JSON format as described below (no Markdowns, no explainations) if tool is being used.

schema:
- If you want to call a tool, respond with arguments as dictionary) :
{{"tool":"<tool_name>","args":{...}}}
- set final=none only if tool is not needed.
- If no tool is needed:
  {{"tool":"none","final":"<your answer>"}}


Avaliable tools:
1. Count_character(text: string, letter:string)->integer: Counts occurance of give character in the word. 
Example: Count_character(text:"hello",letter: "l") counts number of 'l'characters in a given text and 
will give answer as 2.)
2. add_numbers (a: float, b:float)->float: Adds two numbers and returns the result. 
If more or less than 2 number are provided then do not use this tool.
Example: add_numbers('a':2,'b':3) will return 5.
 
"""

# Define the tool call schema
# This schema defines the structure of the tool call JSON that the model will output.
# It includes the tool name, its arguments, and a final answer if no tool is needed.
# This is used to validate the model's output.

In [78]:
from typing import Any, Dict, Literal, Optional
from pydantic import BaseModel, ValidationError, Field
class ToolCall(BaseModel):
    # Literal[...] is a specialized type hint used to restrict a variable to a specific set of exact values
    tool: Literal["Count_character", "add_numbers", "none"] 
    args: Dict[str, Any] = Field(default_factory=dict) # tool arguments dictionary
    # Dict[str, Any] is a dictionary with string keys and any values
    # Field(default_factory=dict) is a field that is a dictionary with string keys and any values
    # default_factory=dict is a default factory that returns a dictionary with string keys and any values
    # Optional[str] is an optional string   
    final: Optional[str] = None  # final answer if no tool is needed

In [12]:
#defining Count_character tool
def Count_character(text: str, letter: str) -> int:
    return text.lower().count(letter.lower())

In [13]:
#define add_numbers tool
def add_numbers(a: float, b: float) -> float:
    return a + b

# Define the function to get the tool decision
# This function takes a user message and returns a ToolCall object.
# It uses the OpenAI client to call the model and get the response.
# It then validates the response and returns a ToolCall object.
# If the response is not valid JSON, it raises an error.

# ToolCall.model_validate is going to validate the response returned by model
# it is going to use ToolCall schema to validate the response which we defined above

In [79]:
def get_tool_decision(user_query: str) -> ToolCall:
    response_text=call_llm(system_instructions, user_query)
    print("response_text:", response_text)

    try:
        data=json.loads(response_text)
        print("Json:", data)
        return ToolCall.model_validate(data)
    except (json.JSONDecodeError, ValidationError) as e:
        raise RuntimeError(f"Model did not return valid tool JSON.\nRaw:\n{response_text}\n\nError:\n{e}") from e

In [93]:
call=get_tool_decision("How many letter i are there in 'AI is my cup of tea!'?")
call_json = call.model_dump_json()

response_text: {"tool":"Count_character","args":{"text":"AI is my cup of tea!","letter":"i"}}
Json: {'tool': 'Count_character', 'args': {'text': 'AI is my cup of tea!', 'letter': 'i'}}


In [94]:
# TOOLS is a dict and it will give literal value as per the key passsed
# and as model output is validated it must be having values like "Count_character", "add_numbers", "none"

TOOLS = {
    "Count_character": Count_character,
    "add_numbers": add_numbers,
}

In [95]:
call

ToolCall(tool='Count_character', args={'text': 'AI is my cup of tea!', 'letter': 'i'}, final=None)

In [96]:
tool_to_be_used=TOOLS[call.tool]
tool_to_be_used

<function __main__.Count_character(text: str, letter: str) -> int>

In [97]:
#checking values of args
call.args

{'text': 'AI is my cup of tea!', 'letter': 'i'}

In [98]:
#call.args={'text': 'AI is my cup of tea!', 'letter': 'i'}
#**call.args: This uses the double asterisk (dictionary unpacking) operator. 
# It takes the key-value pairs from the call.args dictionary and turns them into keyword arguments for the function
# Is exactly the same as writing:result = Count_character(text='AI is my cup of tea!', letter='i')
results=tool_to_be_used(**call.args)

In [99]:
results

2

In [100]:
# this is the result and which looks correct but model needs to take this result and provide answer in proper wording
# so this result is actually context that are providing to use it for crafting answer

This is the additional code

In [117]:
def get_final_answer(user_msg: str, call_json: str, tool_result: Any) -> str:
    """Ask the model to produce the final user-facing response using tool result."""
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Use the tool result to answer."},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": call_json},
            {"role": "user", "content": f"Tool result: {tool_result}\nNow answer clearly in 1-3 sentences."},
        ],
        temperature=0)
    return resp.choices[0].message.content.strip()

In [118]:
call_json

'{"tool":"Count_character","args":{"text":"AI is my cup of tea!","letter":"i"},"final":null}'

In [119]:
# calling the model again and providing following inputs
# original question that was asked 
# output that model earlire provided
# output of the tool calls 

model_response = get_final_answer("How many letter i are there in 'AI is my cup of tea!'?", call_json, results)
print("model_response: ", model_response)

model_response:  There are 2 instances of the letter 'i' in the phrase "AI is my cup of tea!"


In [120]:
call=get_tool_decision("What is sum of 12.34 and 45.12?")
tool_to_be_used=TOOLS[call.tool]
results=tool_to_be_used(**call.args)
results
model_response = get_final_answer("What is sum of 12.34 and 45.12?", call_json, results)
print("model_response: ", model_response)

response_text: {"tool":"add_numbers","args":{"a":12.34,"b":45.12}}
Json: {'tool': 'add_numbers', 'args': {'a': 12.34, 'b': 45.12}}
model_response:  The sum of 12.34 and 45.12 is approximately 57.46.
